## DIFFERENT APPROACH: SCORE DIFFERENCE IN 50/50 SPLIT
Does that solve the range problems caused by gamma?

In [ ]:
N_BOOTSTRAP = 100
gamma_range = np.linspace(0.1, 2, 40)

score_diff = np.zeros((len(df), N_BOOTSTRAP, len(gamma_range)))

hardmin_diff = np.zeros((len(df), N_BOOTSTRAP))

# NECESSARY TO USE ALL INSTANCES
def clean_distances(z):
    if z[0] == 0:
        return z[1:]
    else:
        return z[0:-1]


for i in range(0, N_BOOTSTRAP):

    # 50/50 split
    sample = df.sample(frac=0.5)
    rest = df.drop(sample.index)

    # USE ALL INSTANCES
    nbrs = NearestNeighbors(n_neighbors=len(sample), algorithm="ball_tree").fit(sample)
    distances1, _ = nbrs.kneighbors(df)
    distances1 = np.apply_along_axis(clean_distances, 1, distances1)
    distances1 = np.square(distances1)

    nbrs = NearestNeighbors(n_neighbors=len(rest), algorithm="ball_tree").fit(rest)
    distances2, _ = nbrs.kneighbors(df)
    distances2 = np.apply_along_axis(clean_distances, 1, distances2)
    distances2 = np.square(distances2)

    for j, gamma in enumerate(gamma_range):
        sm1 = np.apply_along_axis(softmin_og, 1, distances1, gamma)
        sm2 = np.apply_along_axis(softmin_og, 1, distances2, gamma)

        score_diff[:, i, j] = sm2 - sm1

    # hardmin
    hardmin1 = distances1[:, 1]
    hardmin2 = distances2[:, 1]

    hardmin_diff[:, i] = hardmin2 - hardmin1


In [ ]:
# hardmin has low bias but high variance (not so robust)
# TODO interpretation of that value corect?
np.mean(np.var(hardmin_diff, axis=0))


In [ ]:
# hardmin score # TODO interpretation of that value?
np.var(np.mean(hardmin_diff, axis=0))


In [ ]:
score_diff.shape

In [ ]:
spread = np.var(score_diff, axis=1)

for i in range(len(scores)):
    plt.plot(gamma_range, spread[i], linewidth=0.3)
plt.show()


In [ ]:
# robustness (how much to the scores vary over the bootstrap samples?)
# robustness increases with lower gamma (but high bias)
sns.scatterplot(x = gamma_range, y=np.mean(spread, axis=0))

In [ ]:
avg_score_diff = np.mean(score_diff, axis=1)

for i in range(len(scores)):
    plt.plot(gamma_range, avg_score_diff[i], linewidth=0.3)
plt.show()


In [ ]:
# measure discrimination 
# how good can we distinguish between the scores?
# higher variance is better here to identify the outliers
sns.scatterplot(x = gamma_range, y=np.var(avg_score_diff, axis=0))

In [ ]:
sns.scatterplot(x = np.mean(spread, axis=0), y=np.var(avg_score_diff, axis=0), hue=gamma_range)

## Before:

In [ ]:

nbrs = NearestNeighbors(n_neighbors=2, algorithm="ball_tree").fit(df)
distances, indices = nbrs.kneighbors(df)
df_overview["outlier_score_min"] = np.square(distances[:, 1])
df_overview.sort_values(by="outlier_score_min", ascending=False).head(10)
print(distances.shape)
# distances: Zeile = sample, Spalte 1 = kürzeste Distanz
# Spalte 0 = alles 0
print(indices.shape)
# indices: erste Spalte 0-440; zweite Spalte: index zur kürzesten Distanz
print(distances[0, 1], indices[0])
# quick double check
np.linalg.norm(df.iloc[0, :] - df.iloc[59, :])
print(np.var(df_overview['outlier_score_min']))
display(df_overview[["outlier_score_min"]].describe())
sns.boxplot(df_overview["outlier_score_min"])
plt.show()
sns.histplot(df_overview["outlier_score_min"])
# want it to be centered/dense!

# score_bias_eval function before

In [ ]:
# Compare hardmin with hardmin-bootstrap appraoch by statistics as well as artificial outlier classification

# def score_bias_eval(score, overview_data=df_overview, outlier_frac=OUTLIERS_FRAC):
#     n_outliers = math.ceil(len(df_overview) * outlier_frac)
#     spr = spearmanr(overview_data[score], df_overview["hardmin_bootstrap_score"])
#     hardmin_bootstrap_ranking = df_overview.sort_values(by="hardmin_bootstrap_score", ascending=False).index
#     hardmin_outliers = hardmin_bootstrap_ranking[:n_outliers]
#     score_outlier_ranking = df_overview.sort_values(by=score, ascending=False).index
#     score_outliers = score_outlier_ranking[:n_outliers]

#     accuracy = len(set(hardmin_outliers).intersection(score_outliers)) / n_outliers
#     return spr, accuracy

# spr, acc = score_bias_eval(overview_data=df_overview, score="hardmin_score")

# print(f"Bootstrap Accuracy: {acc:.2}% \nSpearman results: {spr}")